<center><h2>Generating Transcript </h2>
<h4>ABC format Final (Marks)</h4>
</center>

**Table of content**:

>1. It accepts csv with all the courses of a student in a multiple rows (ABC format)
>2. Splits the row into different rows resulting into a dataset of course and marks
>3.  


## 1. Importing Libraries and Reading Files 

In [1]:
from reportlab.pdfgen import canvas
import pandas as pd
from datetime import datetime
import warnings
import os
import re
warnings.filterwarnings("ignore")

In [2]:
from reportlab.platypus import (
    BaseDocTemplate, PageTemplate, Frame, Table, TableStyle,
    Spacer, Paragraph, NextPageTemplate
)

In [3]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.platypus import KeepInFrame, SimpleDocTemplate, Table, TableStyle, Spacer, Paragraph, \
                               KeepTogether, Frame, PageTemplate, PageBreak, Flowable

from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_CENTER, TA_LEFT

In [4]:
from xml.sax.saxutils import escape


In [5]:
styles = getSampleStyleSheet()

In [7]:
# University logo
IMAGE_PATH = r"D:\data\gcu\images\GCU Logo transparent.png"

# Base directories
base_dir   = r"D:\data\gcu\erp\transcript\with marks"
PHOTO_DIR  = os.path.join(base_dir, "photos")          # if you store photos here
output_dir = os.path.join(base_dir, "transcripts_pdf") # output folder for PDFs

# Ensure output folder exists
os.makedirs(output_dir, exist_ok=True)

In [9]:
df_courses = pd.read_csv(os.path.join(base_dir, "diploma_pharma_course_details.csv"),encoding="latin1") 
df_courses.head(5)

,year,course code,course name,formative,summative
0,1,ER20-11P,Practical - Pharmaceutics  Practical,20,80
1,1,ER20-11T,Lecture - Pharmaceutics  Theory,20,80
2,1,ER20-12P,Practical - Pharmaceutical Chemistry  Practical,20,80
3,1,ER20-12T,Lecture - Pharmaceutical Chemistry  Theory,20,80
4,1,ER20-13T,Lecture - Pharmacognosy  Theory,20,80


In [15]:
df_courses["course name"] = df_courses["course name"].astype(str).str.replace(r"\x96", "", regex=True)
df_courses["course name"] = df_courses["course name"].apply(lambda x: x.split('-')[1])

In [28]:
df_courses.columns = df_courses.columns.str.strip()
df_courses.head(5)

,year,course code,course name,formative,summative
0,1,ER20-11P,Pharmaceutics Practical,20,80
1,1,ER20-11T,Pharmaceutics Theory,20,80
2,1,ER20-12P,Pharmaceutical Chemistry Practical,20,80
3,1,ER20-12T,Pharmaceutical Chemistry Theory,20,80
4,1,ER20-13T,Pharmacognosy Theory,20,80


In [29]:
df_courses.columns

Index(['year', 'course code', 'course name', 'formative', 'summative'], dtype='object')

In [40]:
df_diploma = pd.read_excel(os.path.join(base_dir, "diploma.xls")) 
df_diploma.head(3)

,S.No.,Student Code,Admission Code,Student Name,First Name,Program Name,CGPA,Batch Name,Formative (20),Summative (80),...,Summative (80).18,Total Marks.18,Formative (20).19,Summative (80).19,Total Marks.19,Formative (20).20,Summative (80).20,Total Marks.20,Total.1,SGPA.1
0,1,2314021001,547,ABDUL KADER,ABDUL KADER,Diploma in Pharmacy (D. Pharm) - Guwahati,0.0,0,16,65,...,66.0,84,9.0,53.0,62,9.0,42.0,51,793,0.0
1,2,2314021002,548,ALVAN NEZER KHARKONGOR,ALVAN NEZER KHARKONGOR,Diploma in Pharmacy (D. Pharm) - Guwahati,0.0,0,18,73,...,64.0,83,13.0,59.0,72,12.0,42.0,54,797,0.0
2,3,2314021003,598,AMINUL ISLAM,AMINUL ISLAM,Diploma in Pharmacy (D. Pharm) - Guwahati,0.0,0,16,58,...,62.0,80,10.0,45.0,55,13.0,27.0,40,709,0.0


In [23]:
#df_diploma.columns

Index(['S.No.', 'Student Code', 'Admission Code', 'Student Name', 'First Name',
       'Program Name', 'CGPA', 'Batch Name', 'Formative (20)',
       'Summative (80)', 'Total Marks', 'Formative (20).1', 'Summative (80).1',
       'Total Marks.1', 'Formative (20).2', 'Summative (80).2',
       'Total Marks.2', 'Formative (20).3', 'Summative (80).3',
       'Total Marks.3', 'Formative (20).4', 'Summative (80).4',
       'Total Marks.4', 'Formative (20).5', 'Summative (80).5',
       'Total Marks.5', 'Formative (20).6', 'Summative (80).6',
       'Total Marks.6', 'Formative (20).7', 'Summative (80).7',
       'Total Marks.7', 'Formative (20).8', 'Summative (80).8',
       'Total Marks.8', 'Formative (20).9', 'Summative (80).9',
       'Total Marks.9', 'Total', 'SGPA', 'Unnamed: 40', 'Formative (20).10',
       'Summative (80).10', 'Total Marks.10', 'Formative (20).11',
       'Summative (80).11', 'Total Marks.11', 'Formative (20).12',
       'Summative (80).12', 'Total Marks.12', 'Formati

In [41]:
cols_to_delete = ['S.No.', 'Admission Code','First Name','CGPA','SGPA','SGPA.1']
df_diploma = df_diploma.drop(columns = cols_to_delete)
df_diploma.columns = df_diploma.columns.str.strip()
df_diploma.columns

Index(['Student Code', 'Student Name', 'First Name', 'Program Name',
       'Batch Name', 'Formative (20)', 'Summative (80)', 'Total Marks',
       'Formative (20).1', 'Summative (80).1', 'Total Marks.1',
       'Formative (20).2', 'Summative (80).2', 'Total Marks.2',
       'Formative (20).3', 'Summative (80).3', 'Total Marks.3',
       'Formative (20).4', 'Summative (80).4', 'Total Marks.4',
       'Formative (20).5', 'Summative (80).5', 'Total Marks.5',
       'Formative (20).6', 'Summative (80).6', 'Total Marks.6',
       'Formative (20).7', 'Summative (80).7', 'Total Marks.7',
       'Formative (20).8', 'Summative (80).8', 'Total Marks.8',
       'Formative (20).9', 'Summative (80).9', 'Total Marks.9', 'Total',
       'Unnamed: 40', 'Formative (20).10', 'Summative (80).10',
       'Total Marks.10', 'Formative (20).11', 'Summative (80).11',
       'Total Marks.11', 'Formative (20).12', 'Summative (80).12',
       'Total Marks.12', 'Formative (20).13', 'Summative (80).13',
       'T

In [54]:
#df_diploma = df_diploma.drop(columns = ['First Name'])

### 2 Preprocessing the courses and assessment columns

In [30]:
def process_course(df: pd.DataFrame, df_courses: pd.DataFrame) -> pd.DataFrame:
    """
    Replace repeated column groups like ['Formative (20)', 'Summative (80)', 'Total Marks']
    with ['Formative {course_code}', 'Summative {course_code}', 'Total Marks {course_code}']
    using course codes from df_courses.
    """
    df = df.copy()
    rename_map = {}

    # Get the list of course codes
    course_codes = df_courses["course code"].tolist()

    # Base patterns for the first course (no suffix)
    base_patterns = [
        ("Formative (20)", "Formative"),
        ("Summative (80)", "Summative"),
        ("Total Marks", "Total Marks")
    ]

    # --- Handle the first set (no .1 suffix)
    if len(course_codes) > 0:
        code = str(course_codes[0])
        for old, new_base in base_patterns:
            new_name = f"{new_base} {code}"
            if old in df.columns:
                rename_map[old] = new_name

    # --- Handle subsequent sets (with .1, .2, etc.)
    for i, code in enumerate(course_codes[1:], start=1):
        suffix = f".{i}"
        for old, new_base in base_patterns:
            old_col = f"{old}{suffix}"
            new_col = f"{new_base} {code}"
            if old_col in df.columns:
                rename_map[old_col] = new_col

    # Rename columns
    df = df.rename(columns=rename_map)

    print(f"✅ Renamed {len(rename_map)} columns successfully.")
    return df


In [42]:
df_processed = process_course(df_diploma, df_courses)
df_processed.head(3)

✅ Renamed 63 columns successfully.


,Student Code,Student Name,First Name,Program Name,Batch Name,Formative ER20-11P,Summative ER20-11P,Total Marks ER20-11P,Formative ER20-11T,Summative ER20-11T,...,Formative ER20-25P,Summative ER20-25P,Total Marks ER20-25P,Formative ER20-25T,Summative ER20-25T,Total Marks ER20-25T,Formative ER20-26T,Summative ER20-26T,Total Marks ER20-26T,Total.1
0,2314021001,ABDUL KADER,ABDUL KADER,Diploma in Pharmacy (D. Pharm) - Guwahati,0,16,65,81,13,44,...,18.0,66.0,84,9.0,53.0,62,9.0,42.0,51,793
1,2314021002,ALVAN NEZER KHARKONGOR,ALVAN NEZER KHARKONGOR,Diploma in Pharmacy (D. Pharm) - Guwahati,0,18,73,91,16,60,...,19.0,64.0,83,13.0,59.0,72,12.0,42.0,54,797
2,2314021003,AMINUL ISLAM,AMINUL ISLAM,Diploma in Pharmacy (D. Pharm) - Guwahati,0,16,58,74,12,50,...,18.0,62.0,80,10.0,45.0,55,13.0,27.0,40,709


In [43]:
df_processed.columns

Index(['Student Code', 'Student Name', 'First Name', 'Program Name',
       'Batch Name', 'Formative ER20-11P', 'Summative ER20-11P',
       'Total Marks ER20-11P', 'Formative ER20-11T', 'Summative ER20-11T',
       'Total Marks ER20-11T', 'Formative ER20-12P', 'Summative ER20-12P',
       'Total Marks ER20-12P', 'Formative ER20-12T', 'Summative ER20-12T',
       'Total Marks ER20-12T', 'Formative ER20-13T', 'Summative ER20-13T',
       'Total Marks ER20-13T', 'Formative ER20-13P', 'Summative ER20-13P',
       'Total Marks ER20-13P', 'Formative ER20-14P', 'Summative ER20-14P',
       'Total Marks ER20-14P', 'Formative ER20-14T', 'Summative ER20-14T',
       'Total Marks ER20-14T', 'Formative ER20-15P', 'Summative ER20-15P',
       'Total Marks ER20-15P', 'Formative ER20-15T', 'Summative ER20-15T',
       'Total Marks ER20-15T', 'Total', 'Unnamed: 40', 'Formative ER20-21P',
       'Summative ER20-21P', 'Total Marks ER20-21P', 'Formative ER20-21T',
       'Summative ER20-21T', 'Total Mar

In [44]:
df_processed = df_processed.rename(columns={'Total':'Total Year 1','Total.1':'Total Year 2'})
df_processed = df_processed.loc[:, ~df_processed.columns.str.contains('^Unnamed')]
#df_renamed

In [50]:
df_processed.columns

Index(['Student Code', 'Student Name', 'First Name', 'Program Name',
       'Batch Name', 'Formative ER20-11P', 'Summative ER20-11P',
       'Total Marks ER20-11P', 'Formative ER20-11T', 'Summative ER20-11T',
       'Total Marks ER20-11T', 'Formative ER20-12P', 'Summative ER20-12P',
       'Total Marks ER20-12P', 'Formative ER20-12T', 'Summative ER20-12T',
       'Total Marks ER20-12T', 'Formative ER20-13T', 'Summative ER20-13T',
       'Total Marks ER20-13T', 'Formative ER20-13P', 'Summative ER20-13P',
       'Total Marks ER20-13P', 'Formative ER20-14P', 'Summative ER20-14P',
       'Total Marks ER20-14P', 'Formative ER20-14T', 'Summative ER20-14T',
       'Total Marks ER20-14T', 'Formative ER20-15P', 'Summative ER20-15P',
       'Total Marks ER20-15P', 'Formative ER20-15T', 'Summative ER20-15T',
       'Total Marks ER20-15T', 'Total Year 1', 'Formative ER20-21P',
       'Summative ER20-21P', 'Total Marks ER20-21P', 'Formative ER20-21T',
       'Summative ER20-21T', 'Total Marks ER20-

In [49]:
df_processed.shape

(64, 70)

### 2. Convert rows (student with multiple subjects) into multiple rows for a single student

In [51]:
def process_marks_long_format(df, df_courses=None):
    """
    Convert wide-format student marks DataFrame into long-format.
    Optionally merges with df_courses for course details.

    Parameters
    ----------
    df : pd.DataFrame
        The wide-format DataFrame with columns like 'Formative ER20-11P', 'Summative ER20-11P', etc.
    df_courses : pd.DataFrame, optional
        DataFrame containing course info with columns ['year', 'course code', 'course name', 'formative', 'summative'].

    Returns
    -------
    pd.DataFrame
        Long-format DataFrame with columns:
        ['Student Code', 'Student Name', 'First Name', 'Program Name', 'Batch Name',
         'Course Code', 'Assessment Type', 'Marks', 'course name', 'year', ... (if merged)]
    """

    # Identify base student info columns
    id_vars = ['Student Code', 'Student Name', 'First Name', 'Program Name', 'Batch Name']

    # Melt the rest into long format
    df_long = df.melt(id_vars=id_vars, var_name='Assessment', value_name='Marks')

    # Extract course code (e.g., ER20-11P)
    df_long['Course Code'] = df_long['Assessment'].apply(
        lambda x: re.search(r'(ER\d{2}-\d{2}[A-Z]?)', x).group(1) if re.search(r'(ER\d{2}-\d{2}[A-Z]?)', x) else None
    )

    # Extract assessment type
    df_long['Assessment Type'] = df_long['Assessment'].apply(
        lambda x: 'Formative' if 'Formative' in x
        else ('Summative' if 'Summative' in x
        else ('Total Marks' if 'Total Marks' in x else None))
    )

    # Keep only valid rows
    df_long = df_long.dropna(subset=['Course Code', 'Assessment Type'])

    # Reorder columns
    df_long = df_long[
        ['Student Code', 'Student Name', 'First Name', 'Program Name', 'Batch Name',
         'Course Code', 'Assessment Type', 'Marks']
    ]

    # Merge with df_courses if provided
    if df_courses is not None and not df_courses.empty:
        df_long = df_long.merge(df_courses, left_on='Course Code', right_on='course code', how='left')

    return df_long


In [53]:
df_long = process_marks_long_format(df_processed, df_courses)
df_long.head()

,Student Code,Student Name,First Name,Program Name,Batch Name,Course Code,Assessment Type,Marks,year,course code,course name,formative,summative
0,2314021001,ABDUL KADER,ABDUL KADER,Diploma in Pharmacy (D. Pharm) - Guwahati,0,ER20-11P,Formative,16.0,1,ER20-11P,Pharmaceutics Practical,20,80
1,2314021002,ALVAN NEZER KHARKONGOR,ALVAN NEZER KHARKONGOR,Diploma in Pharmacy (D. Pharm) - Guwahati,0,ER20-11P,Formative,18.0,1,ER20-11P,Pharmaceutics Practical,20,80
2,2314021003,AMINUL ISLAM,AMINUL ISLAM,Diploma in Pharmacy (D. Pharm) - Guwahati,0,ER20-11P,Formative,16.0,1,ER20-11P,Pharmaceutics Practical,20,80
3,2314021004,ANJAN KR NATH,ANJAN KR NATH,Diploma in Pharmacy (D. Pharm) - Guwahati,0,ER20-11P,Formative,0.0,1,ER20-11P,Pharmaceutics Practical,20,80
4,2314021005,ARIF AHMED,ARIF AHMED,Diploma in Pharmacy (D. Pharm) - Guwahati,0,ER20-11P,Formative,16.0,1,ER20-11P,Pharmaceutics Practical,20,80


In [61]:
len(df_long[(df_long['Student Code']==2314021001) & (df_long['year']==1)])

30

In [22]:
#serial_no = REGN_NO

In [62]:
#summary_df.head()

In [63]:
#df_processed.columns

In [26]:
#if "NCRF_LEVEL" not in df_processed.columns:  
#    df_processed["NCRF_LEVEL"] = ""


#df_processed["NCRF_LEVEL"] = df_processed["NCRF_LEVEL"].fillna(" ")
#df_processed = df_processed.dropna(subset=['SUB_NAME', 'SUB_CODE'], how='all')

In [27]:
#one_student.info()

#### Just trying to print a students semester results

In [28]:
# Sunit dey
#one_student = df_processed[['RROLL','COURSE_NAME','STREAM','CNAME','SEM','SUB_NAME','SUB_CODE', 'GRADE']]
#one_student = one_student.dropna(subset=['SUB_NAME', 'SUB_CODE'], how='all')
#one_student_filtered = one_student[(one_student['RROLL'].astype(str) == "2312004006") & (one_student['SEM'].str.strip() == "I")]
#one_student_filtered

In [64]:
# ---------------- SAFE HELPERS ----------------
def safe_round(value, ndigits=2, default=0):
    try:
        return round(float(value), ndigits)
    except (ValueError, TypeError):
        return default

def remove_decimal(value):
    try:
        val = float(value)
        if val.is_integer():
            return int(val)
        return round(val, 2)
    except Exception:
        return value
        
# --- Safe paragraph wrapper ---
def safe_paragraph(text, style):
    """Wrap text safely in a Paragraph, escaping XML chars"""
    if text is None:
        text = ""
    return Paragraph(escape(str(text)), style)

# New helper function to clean up problematic characters
def clean_text_for_reportlab(text):
    if text is None:
        return ""
    # Convert to string and replace common problematic characters (e.g., non-breaking spaces)
    text = str(text).replace('\xa0', ' ').replace('\u2013', '-').strip()
    # If the issue is with special UTF-8 characters, you might need a more aggressive filter:
    # return text.encode('ascii', 'ignore').decode('ascii')
    
    return text

def safe_text(val, default=""):
    """Ensure text is safe for ReportLab Paragraphs/drawString."""
    if pd.isna(val) or val is None:
        return default
    return str(val)

def safe_number_text(val, default=""):
    """Ensure numbers don’t come in scientific notation."""
    try:
        return str(int(float(val)))
    except Exception:
        return default

In [65]:
def find_photo(photo_dir, enrollment_no):
    """
    Find the student's photo file based on enrollment number.
    Supports .jpg, .jpeg, and .png extensions.
    
    Returns:
        str | None: Full path to photo if found, else None.
    """
    extensions = ["jpg", "jpeg", "png"]
    for ext in extensions:
        candidate = os.path.join(photo_dir, f"{enrollment_no}.{ext}")
        if os.path.exists(candidate):
            return candidate
    return None


## 3. Draw Header

In [67]:
def draw_header_with_photo(canvas, doc, student_data):
    width, height = A4
    usable_width = width - doc.leftMargin - doc.rightMargin

    enrollment_no = safe_text(student_data.iloc[0].get("Student Code", ""))
    student_name = safe_text(student_data.iloc[0].get("Student Name", ""))
    #apaar_id = safe_number_text(student_data.iloc[0].get("ABC_ACCOUNT_ID", ""))
    apaar_id = 0
    #serial_no = safe_number_text(student_data.iloc[0].get("REGN_NO", ""))
    serial_no =0
    course_name = safe_text(student_data.iloc[0].get("course name", ""))
    #level = safe_text(student_data.iloc[0].get("NCRF_LEVEL", ""))
    level =0

    # --- Logo ---
    try:
        canvas.drawImage(IMAGE_PATH, 50, height - 140, width=80, height=80, mask='auto')
    except Exception:
        pass

    # --- Photo ---
    photo_file = find_photo(PHOTO_DIR, enrollment_no)

    if not photo_file:
        print(f"⚠️ No photo found for {student_name} ({enrollment_no})")
    try:
        if photo_file:
            canvas.drawImage(photo_file, 450, height - 140, width=70, height=80, mask='auto')
    except Exception:
        pass

    # --- Titles ---
    canvas.setFont("Times-Roman", 18)
    canvas.drawCentredString(width / 2, height - 60, "Girijananda Chowdhury University")
    canvas.setFont("Times-Roman", 14)
    canvas.drawCentredString(width / 2, height - 85, "TRANSCRIPT")
    canvas.setFont("Times-Roman", 12)
    canvas.drawCentredString(width / 2, height - 100, course_name)
    canvas.drawCentredString(width / 2, height - 115, "2023-2025")

    # --- Student data table ---
    std_data = [
        ["APAAR ID", ":", apaar_id, "Serial Number : " + serial_no],
        ["Enrollment No.", ":", enrollment_no, ""],
        ["Name", ":", student_name, ""],
        ["NCrF/NHEQF Level", ":", level, ""],
    ]
    std_table = Table(std_data, colWidths=[120, 10, 200, 150])
    std_table.setStyle(TableStyle([
        ("FONTNAME", (0,0), (-1,-1), "Helvetica"),
        ("FONTSIZE", (0,0), (-1,-1), 10),
        ("ALIGN", (0,0), (0,-1), "LEFT"),
        ("VALIGN", (0,0), (-1,-1), "MIDDLE"),
        ("TOPPADDING", (0,0), (-1,-1), 1),
        ("BOTTOMPADDING", (0,0), (-1,-1), 1),
    ]))
    table_width, table_height = std_table.wrap(0, 0)
    std_table.drawOn(canvas, 50, height - 150 - table_height)


### 3.1 Styles

In [68]:
left_sem_style = ParagraphStyle(
    name="LeftSemesterHeading",
    parent=styles['Heading4'],
    alignment=TA_LEFT,
    leftIndent=-30,   # moves 15 points left of the normal margin
    firstLineIndent=0,
    fontName="Helvetica",   # <--- this removes italics
)

In [69]:
def format_subject_name(text, max_len=45):
    """
    Returns a Paragraph with adaptive font size.
    If the subject name is too long, reduce font size.
    """
    styles = getSampleStyleSheet()
    subject_style = styles["Normal"]
    subject_style.fontName = "Times-Roman"
    subject_style.leading = 11
    subject_style.alignment = TA_LEFT

    if len(str(text)) > max_len:  
        subject_style.fontSize = 8   # shrink if too long
        subject_style.leading = 10
    elif len(str(text)) > max_len * 1.5:  
        subject_style.fontSize = 7   # extra shrink
        subject_style.leading = 9
    else:
        subject_style.fontSize = 9   # normal case
    safe_text = escape(str(text))  # prevent invalid XML
    return Paragraph(str(text), subject_style)

# --- Safe paragraph wrapper ---
def safe_paragraph(text, style):
    """Wrap text safely in a Paragraph, escaping XML chars"""
    if text is None:
        text = ""
    return Paragraph(escape(str(text)), style)

def safe_number(x):
    """Ensure numeric values are safe for ReportLab (no NaN)."""
    try:
        if pd.isna(x):
            return 0
        return int(x) if float(x).is_integer() else round(float(x), 2)
    except Exception:
        return 0

In [70]:
df_long.columns

Index(['Student Code', 'Student Name', 'First Name', 'Program Name',
       'Batch Name', 'Course Code', 'Assessment Type', 'Marks', 'year',
       'course code', 'course name', 'formative', 'summative'],
      dtype='object')

## 4. Generate PDFs

In [71]:
def generate_pdf(student_id, student_data, report_date):
    width, height = A4
    #report_date = '25-09-2025'
    filename = os.path.join(output_dir, f"Transcript_{student_data.iloc[0]['CNAME']}.pdf")
    doc = BaseDocTemplate(filename, pagesize=A4)
    doc.showFooter = True

    # --- Styles ---
    styles = getSampleStyleSheet()
    subject_style = styles["Normal"]
    subject_style.fontName = "Times-Roman"
    subject_style.fontSize = 9
    subject_style.leading = 11
    subject_style.alignment = TA_LEFT

    bold_subject_style = ParagraphStyle(
        name="SubjectBold",
        parent=subject_style,
        fontName="Times-Bold",
        fontSize=subject_style.fontSize,
        leading=subject_style.leading,
        alignment=subject_style.alignment
    )

    left_sem_style = ParagraphStyle(
        name="LeftSemesterHeading",
        parent=styles['Heading4'],
        alignment=TA_LEFT,
        leftIndent=-30,
        fontName="Helvetica"
    )

    # --- Frames ---
    frame_first = Frame(doc.leftMargin, doc.bottomMargin, doc.width, doc.height - 140, id='first_frame')
    frame_later = Frame(doc.leftMargin, doc.bottomMargin, doc.width, doc.height + 50, id='later_frame')


    # --- Page templates ---
    first_page_template = PageTemplate(
        id='FirstPage',
        frames=[frame_first],
        onPage=lambda c, d: draw_header_with_photo(c, d, student_data)
        # Notice: no onPageEnd here, so no footer/signature on Page 1
    )
    
    middle_page_template = PageTemplate(
        id='MiddlePages',
        frames=[frame_later],
        onPageEnd=lambda c, d: draw_footer(c, d, report_date)
    )
    
    doc.addPageTemplates([first_page_template, middle_page_template])

    # --- Table style ---
    table_style = TableStyle([
        ("GRID", (0,0), (-1,-1), 0.5, colors.grey),
        ("FONTNAME", (0,0), (-1,-1), "Times-Roman"),
        ("FONTSIZE", (0,0), (-1,-1), 9),
        ("FONTNAME", (0,0), (-1,0), "Times-Bold"),
        ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),
        ("ALIGN", (2,1), (-1,-1), "CENTER"),
    ])

    elements = []

    # --- Group by SEM ---
    for i, (year, year_data) in enumerate(student_data.groupby("year")):
        if i == 1:
            elements.append(NextPageTemplate('MiddlePages'))

        # Build table rows
        data = [["Sub Code", "Subject/Papers", "Credit", "Grade", "Grade Points", "Credit Points"]]
        
        """
        ['Student Code', 'Student Name', 'First Name', 'Program Name',
       'Batch Name', 'Course Code', 'Assessment Type', 'Marks', 'year',
       'course code', 'course name', 'formative', 'summative']
        """
        
        
        
        

        for _, row in year_data.iterrows():
            if row[["course code","course name",'formative','summative','Marks']].isna().all():
                continue

            # Clean + escape subject code and subject name
            sub_code = str(row.get("course code", "")).replace("/", "_")
            sub_name = format_subject_name(row.get("course name", ""))

            data.append([
                escape(sub_code),
                sub_name,
                safe_number(row.get("formative", 0)),
                escape(str(row.get("summative", ""))),
                safe_number(row.get("Marks", 0)),
                #safe_number(row.get("CREDIT_POINTS", 0))
            ])

        if len(data) == 1:  # skip empty semesters
            continue

        # SGPA row
        total_marks = safe_number(sem_data["Marks"].sum())
        #total_credit_points = safe_number(sem_data["CREDIT_POINTS"].sum())
        #sgpa = round(total_credit_points / total_credits, 2) if total_credits else 0

        total_row = [
            #Paragraph("Total", bold_subject_style),
            #Paragraph(f"SGPA : {sgpa}", bold_subject_style),
            Paragraph(f"Total : {total_marks}", bold_subject_style),
            #safe_number(total_credits),
            "",
            "",
            #safe_number(total_credit_points)
        ]
        data.append(total_row)

        colWidths = [65, 220, 45, 45, 70, 65]
        heading = Paragraph(f"<b>Semester {sem}</b>", left_sem_style)

        table = Table(data, colWidths=colWidths, repeatRows=1)
        table.setStyle(table_style)

        elements.append(KeepTogether([heading, Spacer(1, 6), table]))
        elements.append(Spacer(1, 12))

    # --- Summary section ---
    #try:
    #    cgpa = round(float(student_data["CGPA"].iloc[-1]), 2)
    #except Exception:
    #    cgpa = 0.0

    percentage = round(total_marks * 10, 2)
    #total_credit_points = safe_number(student_data["CREDIT_POINTS"].sum())
    #total_credit = safe_number(student_data["CREDIT"].sum())
    #result = escape(str(student_data.iloc[0].get("RESULT", "")))

    summary_data = [
        ["RESULT", f": PASS", "", "percentage", f": {percentage}"],
        #["CGPA", f": {cgpa}", "", "Grand Total Credit", f": {total_credit}"],
        #["Percentage", f": {percentage}", "", "", ""]
    ]

    usable_width = width - doc.leftMargin - doc.rightMargin
    colWidths = [0.12 * usable_width, 0.12 * usable_width, 0.30 * usable_width,
                 0.28 * usable_width, 0.28 * usable_width]

    summary_table = Table(summary_data, colWidths=colWidths, hAlign='LEFT')
    summary_table.setStyle(TableStyle([
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (1, 0), (1, -1), 'Helvetica-Bold'),
        ('FONTNAME', (4, 0), (4, -1), 'Helvetica-Bold'),
    ]))

    elements.append(summary_table)
    elements.append(Spacer(1, 20))

    # --- Build ---
    elements = [x for x in elements if x not in [None, "", []]]
    try:
        doc.build(elements, canvasmaker=NumberedCanvas)
        print(f"✅ PDF generated: {filename}")
    except Exception as e:
        print(f"❌ PDF build failed: {e}")


In [72]:
def remove_decimal(value):
    try:
        val = float(value)
        if val.is_integer():
            return int(val)
        return round(val, 2)
    except Exception:
        return value

def ensure_row(*cells):
    return list(cells)



In [73]:
def draw_footer(canvas, doc, date_value):
    # Skip footer on first page (page numbers are still handled by NumberedCanvas)
    #if doc.page == 1:
    #    return
    
    width, height = A4
    canvas.setFont("Helvetica", 10)
    canvas.drawString(50, 60, f"Date : {date_value}")
    canvas.drawRightString(width - 55, 70, "Controller of Examination")
    canvas.drawRightString(width - 40, 60, "Girijananda Chowdhury University")


In [74]:
def draw_first_page(canvas, doc):
    canvas.saveState()
    canvas.setFont("Times-Bold", 12)
    canvas.drawCentredString(doc.width/2.0 + doc.leftMargin, doc.height + doc.topMargin - 20, "Your Header for First Page")
    canvas.restoreState()

def draw_later_pages(canvas, doc):
    canvas.saveState()
    canvas.setFont("Times-Roman", 9)
    canvas.drawRightString(doc.width + doc.leftMargin, 20, f"Page {doc.page}")
    canvas.restoreState()


In [75]:
class SetFooterFlag(Flowable):
    """
    Flowable used to indicate that the footer should start appearing
    from this point onward.
    """
    def __init__(self, value=True):
        super().__init__()
        self.value = value

    def draw(self):
        # Store the flag on the doctemplate.
        # draw_footer() will read this flag.
        self._doctemplate.showFooter = self.value

In [76]:
class NumberedCanvas(canvas.Canvas):
    def __init__(self, *args, **kwargs):
        super(NumberedCanvas, self).__init__(*args, **kwargs)
        self._saved_page_states = []

    def showPage(self):
        self._saved_page_states.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        """Add total page count to each page."""
        num_pages = len(self._saved_page_states)
        for state in self._saved_page_states:
            self.__dict__.update(state)
            self.draw_page_number(num_pages)
            super(NumberedCanvas, self).showPage()
        super(NumberedCanvas, self).save()

    def draw_page_number(self, page_count):
        page = f"Page {self._pageNumber} of {page_count}"
        self.setFont("Helvetica", 9)
        width, height = A4
        #self.drawRightString(width - 40, 30, page)  # bottom-right corner
        self.drawCentredString(width / 2.0, 30, page)  # bottom-center

In [77]:
def generate_pdf_onepage(student_id, student_data, report_date):
    """
    Generate a single-page transcript (no page numbers, but with footer).
    """
    width, height = A4
    #report_date ="27-09-2025"
    filename = os.path.join(output_dir, f"Transcript_{student_data.iloc[0]['CNAME']}_onepage.pdf")
    doc = BaseDocTemplate(filename, pagesize=A4)

    # --- Styles ---
    styles = getSampleStyleSheet()
    subject_style = styles["Normal"]
    subject_style.fontName = "Times-Roman"
    subject_style.fontSize = 9
    subject_style.leading = 11
    subject_style.alignment = TA_LEFT

    bold_subject_style = ParagraphStyle(
        name="SubjectBold",
        parent=subject_style,
        fontName="Times-Bold",
        fontSize=subject_style.fontSize,
        leading=subject_style.leading,
        alignment=subject_style.alignment
    )

    left_sem_style = ParagraphStyle(
        name="LeftSemesterHeading",
        parent=styles['Heading4'],
        alignment=TA_LEFT,
        leftIndent=-30,
        fontName="Helvetica"
    )

    # --- Frames ---
    frame_main = Frame(
        doc.leftMargin, doc.bottomMargin,
        doc.width, doc.height - 140,
        id="normal_frame"
    )
    

    # --- Page template (only one page, with footer) ---
    single_page_template = PageTemplate(
        id="OnePage",
        frames=[frame_main],
        onPage=lambda c, d: draw_header_with_photo(c, d, student_data),
        onPageEnd=lambda c, d: draw_footer(c, d, report_date)  # always show footer
    )
    doc.addPageTemplates([single_page_template])

    # --- Elements ---
    elements = []

    # Just one semester (or summary) — adapt depending on your case
    for sem, sem_data in student_data.groupby("SEM"):
        # Table header
        data = [["Sub Code", "Subject/Papers", "Credit",
                 "Grade", "Grade Points", "Credit Points"]]

        for _, row in sem_data.iterrows():
            if row[["SUB_CODE", "SUB_NAME", "CREDIT", "GRADE",
                    "GRADE_POINTS", "CREDIT_POINTS"]].isna().all():
                continue

            data.append([
                row.get("SUB_CODE", ""),
                safe_paragraph(row.get("SUB_NAME", ""), subject_style),
                row.get("CREDIT", 0) or 0,
                row.get("GRADE", ""),
                row.get("GRADE_POINTS", 0) or 0,
                row.get("CREDIT_POINTS", 0) or 0
            ])

        if len(data) == 1:
            continue

        total_credits = sem_data["CREDIT"].sum()
        total_credit_points = sem_data["CREDIT_POINTS"].sum()
        sgpa = round(total_credit_points / total_credits, 2) if total_credits else 0

        data.append([
            Paragraph("Total", bold_subject_style),
            Paragraph(f"SGPA : {sgpa}", bold_subject_style),
            total_credits, "", "", total_credit_points
        ])

        table = Table(data, colWidths=[65, 220, 45, 45, 70, 65], repeatRows=1)
        table.setStyle(TableStyle([
            ("GRID", (0,0), (-1,-1), 0.5, colors.grey),
            ("FONTNAME", (0,0), (-1,-1), "Times-Roman"),
            ("FONTSIZE", (0,0), (-1,-1), 9),
            ("FONTNAME", (0,0), (-1,0), "Times-Bold"),
            ("FONTNAME", (0,-1), (-1,-1), "Times-Bold"),
            ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),
            ("ALIGN", (2,1), (-1,-1), "CENTER"),
        ]))

        elements.append(Paragraph(f"<b>Semester {sem}</b>", left_sem_style))
        elements.append(Spacer(1, 6))
        elements.append(table)
        elements.append(Spacer(1, 12))

    # --- Summary ---
    try:
        cgpa = round(student_data["CGPA"].iloc[-1], 2)
    except Exception:
        cgpa = 0
    percentage = round(cgpa * 10, 2)
    total_credit_points = int(student_data["CREDIT_POINTS"].sum())
    total_credit = int(student_data["CREDIT"].sum())
    result = student_data.iloc[0].get("RESULT", "")

    summary_data = [
        ["RESULT", f": {result}", "", "Grand Total Credit Points", f": {total_credit_points}"],
        ["CGPA", f": {cgpa}", "", "Grand Total Credit", f": {total_credit}"],
        ["Percentage", f": {percentage}", "", "", ""]
    ]

    colWidths = [0.12 * (width - doc.leftMargin - doc.rightMargin),
                 0.12 * (width - doc.leftMargin - doc.rightMargin),
                 0.30 * (width - doc.leftMargin - doc.rightMargin),
                 0.28 * (width - doc.leftMargin - doc.rightMargin),
                 0.28 * (width - doc.leftMargin - doc.rightMargin)]

    summary_table = Table(summary_data, colWidths=colWidths, hAlign='LEFT')
    summary_table.setStyle(TableStyle([
        ('FONTNAME', (0, 0), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (1, 0), (1, -1), 'Helvetica-Bold'),
        ('FONTNAME', (4, 0), (4, -1), 'Helvetica-Bold'),
    ]))

    elements.append(summary_table)
    elements.append(Spacer(1, 20))

    # --- Build (no page numbering here) ---
    elements = [x for x in elements if x not in [None, "", []]]
    try:
        doc.build(elements)  # 🚨 no canvasmaker, so no page numbers
        print(f"✅ One-page PDF generated: {filename}")
    except Exception as e:
        print(f"❌ PDF build failed: {e}")


### 5.1 Selection - One page or Multiple pages

In [80]:
def generate_pdf_auto(student_id, student_data, report_date):
    """
    Automatically choose one-page or multipage PDF generation
    depending on the number of rows.
    """

    # Heuristic: if total subject rows are small, it fits in one page
    total_subjects = student_data[["course code", "course name"]].notna().sum().max()
    
    # Adjust threshold as per your layout (typically ~25–30 rows fits one A4 page)
    if total_subjects <= 15:
        #print(f"ℹ️ Using ONE-PAGE layout for {student_data.iloc[0]['CNAME']}")
        return generate_pdf_onepage(student_id, student_data, report_date)
    else:
        #print(f"ℹ️ Using MULTI-PAGE layout for {student_data.iloc[0]['CNAME']}")
        return generate_pdf(student_id, student_data, report_date)

### 5.2 Main Generator

In [81]:
# Group and generate PDFs, assigning serial numbers
report_date="27-09-2025"
times = 1
for (student_id, student_data) in df_processed.groupby("Student Code"):
    if times == 1:
        generate_pdf_auto(student_id, student_data, report_date )

KeyError: "None of [Index(['course code', 'course name'], dtype='object')] are in the [columns]"

In [ ]:
# Output is generated at file:///D:/Notebooks/Web%20Development/transcripts_pdf/Transcript_BANASMITA%20KAKATI.pdf

### References:

1. https://www.youtube.com/watch?v=q70xzDG6nls&list=PLjNQtX45f0dR9K2sMJ5ad9wVjqslNBIC0
2. https://github.com/bvalgard/create-pdf-with-python-fpdf2
3. https://www.youtube.com/watch?v=p1_7t2SOCIs
4. https://github.com/liannewriting/YouTube-videos-public/blob/main/generate-reports-with-python-sp500/generate_reports_with_python.ipynb
5. https://pyfpdf.readthedocs.io/en/latest/
6. https://github.com/reingart/pyfpdf/blob/master/docs/Tutorial.md